In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame
import os
from geopy import geocoders
from geopy.geocoders import GoogleV3
from config import gkey

import geopandas as gpd
from shapely import wkt

In [2]:
# Pull in geojson address data from openaddresses
fname = "resources/us_wi_city_of_shorewood-addresses-city.geojson"

# read file to dataframe
gdf = gpd.read_file(fname)

# Verify/Make pandas dataframe
addressdf = pd.DataFrame(gdf)

# Change 'geometry' column datatype from geometry to object
addressdf['geometry'] = addressdf.geometry.apply(lambda x: wkt.dumps(x))

In [3]:
# Check dataframe
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,,,,,,,4e61cfaf52ff5d2f,POINT (-87.8906108999999987 43.0995992000000001)
1,,,,,,,,,9c83f18ffe1f809c,POINT (-87.8902592999999968 43.0995757000000026)
2,,,,,,,,,3733c54efa4bc78a,POINT (-87.8904940000000039 43.0967186999999967)
3,,,,,,,,,49643052518dc32c,POINT (-87.8780856000000057 43.0961432000000002)
4,,,,,,,,,3a8547842872c129,POINT (-87.8779176999999976 43.0928486999999976)
...,...,...,...,...,...,...,...,...,...,...
4056,,,,VACANT LAND,,,,,0b76716f60b5c9a3,POINT (-87.8713424999999972 43.0853966000000028)
4057,,,,VACANT LAND,,,,,fd089dcd8802ceec,POINT (-87.8714528999999942 43.0864368000000013)
4058,,,3600,N LAKE DR,,,,,308e595182701e67,POINT (-87.8722834999999947 43.0848050000000029)
4059,,,3921,N LAKE DR,,,,,adab643e296bb675,POINT (-87.8739951999999960 43.0879789999999971)


In [4]:
# Combine address columns to one column
addressdf['addresses'] = addressdf['number'].str.cat(addressdf['street'] + ", Shorewood, Wisconsin",sep=" ")

# Remove extra strings from the latitude and longitude in the geometry column
addressdf['geometry'] = addressdf['geometry'].str.replace('POINT ', '').str.replace('(', '').str.replace(')', '')
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry,addresses
0,,,,,,,,,4e61cfaf52ff5d2f,-87.8906108999999987 43.0995992000000001,", Shorewood, Wisconsin"
1,,,,,,,,,9c83f18ffe1f809c,-87.8902592999999968 43.0995757000000026,", Shorewood, Wisconsin"
2,,,,,,,,,3733c54efa4bc78a,-87.8904940000000039 43.0967186999999967,", Shorewood, Wisconsin"
3,,,,,,,,,49643052518dc32c,-87.8780856000000057 43.0961432000000002,", Shorewood, Wisconsin"
4,,,,,,,,,3a8547842872c129,-87.8779176999999976 43.0928486999999976,", Shorewood, Wisconsin"
...,...,...,...,...,...,...,...,...,...,...,...
4056,,,,VACANT LAND,,,,,0b76716f60b5c9a3,-87.8713424999999972 43.0853966000000028,"VACANT LAND, Shorewood, Wisconsin"
4057,,,,VACANT LAND,,,,,fd089dcd8802ceec,-87.8714528999999942 43.0864368000000013,"VACANT LAND, Shorewood, Wisconsin"
4058,,,3600,N LAKE DR,,,,,308e595182701e67,-87.8722834999999947 43.0848050000000029,"3600 N LAKE DR, Shorewood, Wisconsin"
4059,,,3921,N LAKE DR,,,,,adab643e296bb675,-87.8739951999999960 43.0879789999999971,"3921 N LAKE DR, Shorewood, Wisconsin"


In [5]:
# Remove un-needed columns
addressdf.drop(['unit', 'number', 'street', 'region', 'city', 'district', 'hash'],axis=1,inplace=True)
addressdf

,id,postcode,geometry,addresses
0,,,-87.8906108999999987 43.0995992000000001,", Shorewood, Wisconsin"
1,,,-87.8902592999999968 43.0995757000000026,", Shorewood, Wisconsin"
2,,,-87.8904940000000039 43.0967186999999967,", Shorewood, Wisconsin"
3,,,-87.8780856000000057 43.0961432000000002,", Shorewood, Wisconsin"
4,,,-87.8779176999999976 43.0928486999999976,", Shorewood, Wisconsin"
...,...,...,...,...
4056,,,-87.8713424999999972 43.0853966000000028,"VACANT LAND, Shorewood, Wisconsin"
4057,,,-87.8714528999999942 43.0864368000000013,"VACANT LAND, Shorewood, Wisconsin"
4058,,,-87.8722834999999947 43.0848050000000029,"3600 N LAKE DR, Shorewood, Wisconsin"
4059,,,-87.8739951999999960 43.0879789999999971,"3921 N LAKE DR, Shorewood, Wisconsin"


In [8]:

fltraddress = addressdf[addressdf['addresses'].str.lower().str.contains('e marion')]
fltraddress

,id,postcode,geometry,addresses
1054,,,-87.8772533000000067 43.0944282999999970,"2601 E MARION ST, Shorewood, Wisconsin"
1415,,,-87.8832625000000007 43.0949152000000026,"2106 E MARION ST, Shorewood, Wisconsin"
1416,,,-87.8834870999999964 43.0949146000000027,"2102 E MARION ST, Shorewood, Wisconsin"
1417,,,-87.8837118000000004 43.0949140000000028,"2028 E MARION ST, Shorewood, Wisconsin"
1418,,,-87.8838989000000055 43.0949134999999970,"2022 E MARION ST, Shorewood, Wisconsin"
...,...,...,...,...
2143,,,-87.8913127999999944 43.0949150000000003,"1520 E MARION ST, Shorewood, Wisconsin"
2144,,,-87.8914663000000047 43.0949152999999967,"1516 E MARION ST, Shorewood, Wisconsin"
2145,,,-87.8916198000000009 43.0949156000000002,"1512 E MARION ST, Shorewood, Wisconsin"
2146,,,-87.8917732999999970 43.0949158000000025,"1508 E MARION ST, Shorewood, Wisconsin"


In [ ]:
# adds_sml = fltraddress.sample(n = 100) 
# adds_sml

In [ ]:
# adds_sml.to_csv (r'static/data/sample/adds_sml.csv', index = True, header=True)

In [9]:
# adds_sml = pd.read_csv("static/data/sample/adds_sml.csv")
def format_coordinates(x):
    coordinates = x.split(' ')
    lat = str(coordinates[1])
    long = str(coordinates[0])
    formatted = lat + ', ' + long
    return formatted
fltraddress['geometry'] = fltraddress['geometry'].apply(format_coordinates)
fltraddress


/Users/datascience/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,postcode,geometry,addresses
1054,,,"43.0944282999999970, -87.8772533000000067","2601 E MARION ST, Shorewood, Wisconsin"
1415,,,"43.0949152000000026, -87.8832625000000007","2106 E MARION ST, Shorewood, Wisconsin"
1416,,,"43.0949146000000027, -87.8834870999999964","2102 E MARION ST, Shorewood, Wisconsin"
1417,,,"43.0949140000000028, -87.8837118000000004","2028 E MARION ST, Shorewood, Wisconsin"
1418,,,"43.0949134999999970, -87.8838989000000055","2022 E MARION ST, Shorewood, Wisconsin"
...,...,...,...,...
2143,,,"43.0949150000000003, -87.8913127999999944","1520 E MARION ST, Shorewood, Wisconsin"
2144,,,"43.0949152999999967, -87.8914663000000047","1516 E MARION ST, Shorewood, Wisconsin"
2145,,,"43.0949156000000002, -87.8916198000000009","1512 E MARION ST, Shorewood, Wisconsin"
2146,,,"43.0949158000000025, -87.8917732999999970","1508 E MARION ST, Shorewood, Wisconsin"


In [10]:
# REPLACE 'space' with 'comma' in the geometry column
# adds_sml['geometry'] = adds_sml['geometry'].str.replace(' ',', ')
# adds_sml

In [11]:
# '''addresses = ["2620 N Drake Ave, Chicago, Illinois 60647",
#          "2622 N Drake Ave, Chicago, Illinois 60647",
#          "2626 N Drake Ave, Chicago, Illinois 60647",
#          "2630 N Drake Ave, Chicago, Illinois 60647",
#          "2648 N Drake Ave, Chicago, Illinois 60647"]'''

In [12]:
# '''df = DataFrame (addresses, columns=['Address'])
# df'''

In [13]:
# *** IF WE ONLY HAVE ADDRESSES AND NEED TO CONVERT TO LAT LONG ***

# API_KEY = os.getenv("API1234")
# g = GoogleV3(api_key=gkey)


# loc_coordinates = []
# loc_address = []
# for address in df.Address:
#     try:
#         inputAddress = address
#         location = g.geocode(inputAddress, timeout=15)
#         loc_coordinates.append((location.latitude, location.longitude))
#         loc_address.append(inputAddress)
#     except Exception as e:
#         print('Error, skipping address...', e)


# df_geocodes = pd.DataFrame({'coordinate':loc_coordinates,'address':loc_address})
# df_geocodes

In [14]:
# GRAB A SERIES OF STREET VIEW IMAGES FROM latlong PROVIDED
import json
# Import google_streetview for the api module
import google_streetview.api
import time

In [18]:
# create a params dict that will be updated with new city each iteration
params = {
    'size': '600x300', # max 640x640 pixels
    'heading': '180',
    'location': '',
    'pitch': '-0.76',
    'fov': '120',
    'key': gkey
}

# Loop through the cities_pd and run a lat/long search for each city
geometry_list = fltraddress['geometry'].to_list()
i = 0
for geometry_el in geometry_list[50:70]:
    
    # update address key value
    params['location'] = geometry_el
    
    # Create a results object
#     results = google_streetview.api.results(params)
    results = google_streetview.api.results([params])
    
    
     

    # Download images to directory 'downloads'
    results.download_links(f"static/data/raw/geo{i}",)
    i += 1
    
    # Wait for 1 second
    #time.sleep(1)


# for index, row in adds_sml.iterrows():
#     # get lat, lng from df
#     geometry = row["geometry"]
    
#     # update address key value
#     params['location'] = f"{geometry}"
    
#     # Create a results object
#     results = google_streetview.api.results(params)

#     # Download images to directory 'downloads'
#     results.download_links('static/data/raw')

In [ ]:
# # *** ORIGINAL PULL THAT WORKS ***
# # Define parameters for street view api
# params = [{
#     'size': '600x300', # max 640x640 pixels
#     'location': '41.9295582,-87.7159139',
#     'heading': '90',
#     'pitch': '-0.76',
#     'key': gkey
# }]

# # Create a results object
# results = google_streetview.api.results(params)

# # Download images to directory 'downloads'
# results.download_links('static/data/raw')

In [ ]:
## PRACTICING IMAGE PRE-PROCESSING
'''
# load and show an image with Pillow

# load the image
image = Image.open('static/data/test/brickornot.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()'''

In [ ]:
'''# load all images in a directory
loaded_images = list()
for filename in listdir('images'):
	# load image
	img_data = image.imread('images/' + filename)
	# store loaded image
	loaded_images.append(img_data)
	print('> loaded %s %s' % (filename, img_data.shape))'''


In [ ]:
%matplotlib inline
import PIL
from PIL import Image
from os import listdir
from matplotlib import image
from matplotlib import pyplot

# example of saving a grayscale version of a loaded image
from PIL import Image
# load the image
image = Image.open('static/data/raw/gsv_0.jpg')
# convert the image to grayscale
gs_image = image.convert(mode='L')
# report the size of the image
print(image.size)
# create a thumbnail and preserve aspect ratio
image.thumbnail((100,100))
# report the size of the thumbnail
print(image.size)
# save in jpeg format
gs_image.save('static/data/test_convert/gsv_0.jpg')

In [ ]:
# load the image again and show it
image2 = Image.open('static/data/test_convert/gsv_0.jpg')
# show the image
image2.show()

In [ ]:
'''# save as PNG format
image.save('opera_house.png', format='PNG')
# load the image again and inspect the format
image2 = Image.open('opera_house.png')
print(image2.format)'''